In [9]:
from bokeh.models import Legend, LegendItem, LabelSet, ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
from bokeh.transform import factor_cmap
import pandas as pd


from bokeh.io.export import export_png
from bokeh.io import output_notebook
from selenium import webdriver

In [2]:
# Dados
df = pd.DataFrame({
    "Category": ['Item1','Item2','Item3','Item4','Item5','Item6','Item7','Item8','Item9'],
    "Regime1_Value": [0, 841.77, 1140.52, 0, 674.25, 620.85, 0, 378.72, 0],
    "Regime1_State": ["D", "D", "D", "D", "D", "D", "D", "D", "D"],
    "Regime2_Value": [546.12, 208.61, 93.11, 661.63, 172.70, 53.40, 453.53, 169.86, 265.67],
    "Regime2_State": ["C", "C", "A", "C", "A", "A", "C", "A", "C"],
    "Regime3_Value": [295.65,183.24,0.00,478.89,293.57,0.00,185.32,72.27,113.04],
    "Regime3_State": ["B", "B", "E", "B", "E", "E", "B", "E", "B"]})

In [3]:
df.head()

,Category,Regime1_Value,Regime1_State,Regime2_Value,Regime2_State,Regime3_Value,Regime3_State
0,Item1,0.00,D,546.12,C,295.65,B
1,Item2,841.77,D,208.61,C,183.24,B
2,Item3,1140.52,D,93.11,A,0.00,E
3,Item4,0.00,D,661.63,C,478.89,B
4,Item5,674.25,D,172.70,A,293.57,E


In [4]:
# Calcular o total para cada categoria
df['Total'] = df['Regime1_Value'] + df['Regime2_Value'] + df['Regime3_Value']

# Arredondar todos os valores para uma casa decimal
df['Regime1_Value'] = df['Regime1_Value'].round(1)
df['Regime2_Value'] = df['Regime2_Value'].round(1)
df['Regime3_Value'] = df['Regime3_Value'].round(1)
df['Total'] = df['Total'].round(1)

# Fonte de dados
df['Regime1_top'] = df['Regime1_Value']
df['Regime2_top'] = df['Regime1_top'] + df['Regime2_Value']
df['Regime3_top'] = df['Regime2_top'] + df['Regime3_Value']
df['Regime1_center'] = df['Regime1_top'] / 2
df['Regime2_center'] = df['Regime1_top'] + df['Regime2_Value'] / 2
df['Regime3_center'] = df['Regime2_top'] + df['Regime3_Value'] / 2

# Filtrar rótulos para valores maiores que zero e definir cor com base no valor
df['Regime1_label'] = df['Regime1_Value'].apply(lambda x: f"{x:.1f}" if x > 0 else "")
df['Regime2_label'] = df['Regime2_Value'].apply(lambda x: f"{x:.1f}" if x > 0 else "")
df['Regime3_label'] = df['Regime3_Value'].apply(lambda x: f"{x:.1f}" if x > 0 else "")

df['Regime1_color'] = df['Regime1_Value'].apply(lambda x: "white" if x == 0 else "black")
df['Regime2_color'] = df['Regime2_Value'].apply(lambda x: "white" if x == 0 else "black")
df['Regime3_color'] = df['Regime3_Value'].apply(lambda x: "white" if x == 0 else "black")

source = ColumnDataSource(df)

In [6]:
# Criar figura
p = figure(x_range=df["Category"], height=800, width=1200, title="Stacked Bar Chart with Totals and Conditional Data Labels (One Decimal Place)")

# Adicionar vbar_stack
renderers = p.vbar_stack(["Regime1_Value", "Regime2_Value", "Regime3_Value"],
                         x="Category",
                         fill_color=[
                             factor_cmap(state, palette=["#CC3333", "#4099C9", "#0076B6", 
"white", "#D65C5C"], factors=["A", "B", "C", "D","E"])
                             for state in ["Regime1_State", "Regime2_State", "Regime3_State"]],
                         line_color="white",
                         width=0.9,
                         source=source)

# Adicionar rótulos de dados centralizados para cada camada
labels1 = LabelSet(x='Category', y='Regime1_center', text='Regime1_label',
                   text_align='center', text_baseline='middle',
                   source=source, text_font_size="10pt", text_color='Regime1_color')

labels2 = LabelSet(x='Category', y='Regime2_center', text='Regime2_label',
                   text_align='center', text_baseline='middle',
                   source=source, text_font_size="10pt", text_color='Regime2_color')

labels3 = LabelSet(x='Category', y='Regime3_center', text='Regime3_label',
                   text_align='center', text_baseline='middle',
                   source=source, text_font_size="10pt", text_color='Regime3_color')

# Adicionar rótulos para os valores totais
total_labels = LabelSet(x='Category', y='Regime3_top', text='Total',
                        text_align='center', text_baseline='bottom',
                        source=source, text_font_size="12pt", text_color='black', y_offset=5)

p.add_layout(labels1)
p.add_layout(labels2)
p.add_layout(labels3)
p.add_layout(total_labels)

# Criar itens da legenda manualmente
legend = Legend(items=[
    LegendItem(label="Regime 1", renderers=[renderers[0]]),
    LegendItem(label="Regime 2", renderers=[renderers[1]]),
    LegendItem(label="Regime 3", renderers=[renderers[2]])
])

# Adicionar a legenda ao gráfico
p.add_layout(legend, 'right')

# Configurações adicionais
p.background_fill_color = "white"
p.xgrid.visible = False
p.ygrid.visible = False

# Mostrar gráfico
show(p)
